# Парсинг Тасс и его лемматизация

In [1]:
import requests
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup

import time
import pandas as pd
import pickle

from tqdm import tqdm_notebook

## 1. Скачиваем ссылки на новости 

Все API-запросы к Тасс можно формировать следующим образом через обычный реквест: 

Если нужны вообще все новости:
'http://tass.ru/api/news/lenta?limit=50&before=1488624568'

Если нужны новости только из 25 раздела - экономика:
'http://tass.ru/api/news/lenta?limit=200&sections%5B%5D=25&unmarked[]=1581998&before=1488624568'

In [2]:
# функция, которая крутит вниз ленту из новостей ТАСС
def tass_lenta(before,limit=200):
    mainpage = 'http://tass.ru/api/news/lenta?limit='+str(limit)+'&before='+str(before)
    response = requests.get(mainpage)
    dic = response.json()
    df = dic['articles']
    w =[[item['title'],item['time'],item['url'],item['is_breaking_news'],item['section']] for item in df]
    return(w)  

``` Что ещё можно достать нахаляву из df 
  {'audio': False,
  'color': '2',
  'flash': False,
  'id': '4070830',
  'is_breaking_news': False,
  'is_online': False,
  'live_text': False,
  'marked': False,
  'photos': False,
  'search_queries': None,
  'section': {'id': '25', 'title': 'Экономика и бизнес', 'url': '/ekonomika'},
  'show_at_common_feed': True,
  'show_at_section_feed': True,
  'slideshow': False
  'time': '1488624568',
  'title': 'Глава ВТБ: курс доллара к концу года может достигнуть 61-62 рублей',
  'topics': None,
  'url': '/ekonomika/4070830',
  'video': False}, ```

In [ ]:
%%time
before = 9999999999999
tass_news = [ ]

for i in tqdm_notebook(range(100000)):
    try:
        # if i%100 == 0: print(i,len(tass_news),'\n')
        current_news = tass_lenta(before)
        reserv_1 = current_news[-1]
        reserv_2 = current_news[-2]
        reserv_3 = current_news[-3]  # ну чтобы уж наверняка! Моя вера в ЗБЧ не даёт мне написать функцию для проверки...

        # Смотрим на пропуски в датах: 
        if reserv_1[1] != None:
            before = reserv_1[1]
        elif reserv_2[1] != None:
            print('Какой-то чувак неправильно запостил новость:','\n',reserv_1)
            print('Будем использовать другую:','\n',reserv_2)
            ttt = datetime.datetime.fromtimestamp(int(reserv_2[1])).strftime('%Y-%m-%d %H:%M:%S')
            print('Время публикации другой новости:', ttt, '\n')
            before = reserv_2[1]
        else:
            print('Редкое событие! Две неверные новости подряд:','\n',reserv_1, '\n', reserv_2)
            print('Будем использовать другую:','\n',reserv_3)
            ttt = datetime.datetime.fromtimestamp(int(reserv_3[1])).strftime('%Y-%m-%d %H:%M:%S')
            print('Время публикации другой новости:', ttt, '\n')
            before = reserv_3[1]
        # Расширяем базу из новостей:
        tass_news.extend(current_news)
    # Качаем новости до первой ошибки!    
    except:
        break

In [ ]:
len(tass_news)

In [ ]:
with open('tass_urls.pickle', 'wb') as f:
    pickle.dump(tass_news, f)

## 2. Качаем текст каждой новости по ссылке 

In [3]:
with open('tass_urls.pickle', 'rb') as f:
    tass_urls = pickle.load(f)

In [4]:
len(tass_urls)

648383

In [8]:
def getPage(url):
    response = requests.get(url)
                                       # выгружает данные по ссылке
    html = response.content            # переводит их в читаемый формат, который можно вывести на экран
    soup = BeautifulSoup(html,"lxml")  # отсекает кучу всяких ненужных вещей и запускает поиск по тегам в html  
    return soup

In [9]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text)) 
# очень крутые регулярные выражения - убирают все скобочки, тэги и прочую ерунду
# очищаю посты от тэгов так, а не через .text из-за разнообразия постов


def PageContent(url):
    a = getPage(url)
    vvv = a.findAll("div", { "class" : "b-material-text__l js-mediator-article" })
    text = vvv[0].findAll('p')
    textwp = [html_stripper(item) for item in text]
    return(textwp)

# Функция для объединения абзацев в текст
def Uniter(vect):
    a = vect[0]
    for item in vect[1:]:
        a = a + item
    return(a)

In [10]:
Uniter(PageContent("http://tass.ru" + tass_urls[0][2]))

'ПАРИЖ, 31 декабря. /Корр. ТАСС Иван Батырев/. Официальный Париж полностью поддерживает резолюцию 2336 Совета Безопасности ООН о перемирии в Сирии и ожидает скорейшего возобновления переговоров о мирном урегулировании конфликта в этой стране. Об этом заявили в субботу\xa0в Министерстве иностранных дел Франции."Единогласное принятие резолюции 2336 свидетельствует о том, какую важность мировое сообщество придает установлению перемирия в Сирии. Франция, со своей стороны, уже давно призывает к этому", - говорится в официальном заявлении французского МИД.Как подчеркнули в министерстве, теперь Париж ожидает того, чтобы "перемирие полностью соблюдалось, дабы сохранить жизни мирного населения и обеспечить полноценный доступ гуманитарной помощи". "Срочно необходимо также, чтобы возобновились политические переговоры под эгидой ООН на базе Женевского коммюнике и соответствующих резолюций Совета Безопасности. Франция примет в этом самое деятельное участие", - заключили в МИД республики.Ранее сегод

In [11]:
# Функция берёт одну конкретную новость и узнаёт про неё кучу полезной информации
def href_grab(unic_predata):
    url = "http://tass.ru" + unic_predata[2]
    
    try:
        precontent = PageContent(url)     # скачали контент страницы 
        count_abz = (len(precontent)) # посчитали количество абзацев
        text = Uniter(precontent)  # слили абзацы в текст
    except Exception:   # на Тасс обычно выскакивают ошибки на интервьюшках + ошибки из-за пустот (см. ниже)
        count_abz = 'Error' 
        text = 'Error'
        print('Error')
    
    # Онормаливаем время
    if unic_predata[1] == None:
        new_time = 'Error'
    else:
        new_time = datetime.datetime.fromtimestamp(int(unic_predata[1])).strftime('%Y-%m-%d %H:%M:%S')
        # Отметим, что время будет записано в формате string

    # Блок для диагностики пустоты в куске с тематикой
    if unic_predata[4]==False:
        cur_id = 'Error'
        cur_title= 'Error'
    else:
        cur_id = unic_predata[4]['id']
        cur_title = unic_predata[4]['title']
    
    infa = [unic_predata[0],text,count_abz,new_time,unic_predata[2],unic_predata[3],cur_title,cur_id]
    #[заголовок, текст, число абзацев, время публикации, ссылка, срочная ли новость, название раздела,
    # id раздела]
    return(infa)

In [12]:
href_grab(tass_urls[1])

['ООП заявила о возможном прекращении диалога с США в случае закрытия ее офиса в Вашингтоне',
 'МОСКВА, 18 ноября. /ТАСС/. Организация освобождения Палестины (ООП) выступила с предупреждением о прекращении отношений с Соединенными Штатами в случае закрытия офиса организации в американской столице. Об этом субботу на своей странице в Twitter\xa0заявил генеральный секретарь Исполкома ООП Саиб Эрикат."ООП официально уведомила США, что прекратит отношения с американской администрацией в любом формате, если будет закрыт офис организации в Вашингтоне", - отметил Эрикат. По его словам, отношения будут заморожены до тех пор, "пока офис ООП вновь не заработает в обычном режиме".Ранее в субботу поступила информация, что Госдепартамент не стал продлевать разрешение на работу в американской столице ООП после попытки обратиться в Международный уголовный суд\xa0с обвинениями против израильских властей.Как сообщил представитель пресс-службы американского внешнеполитического ведомства, в соответствии 

In [13]:
href_grab(tass_urls[-7])

['Ливийский лидер продолжает наступление на восставшие города',
 'РАБАТ, 25 февраля. /Корр. ИТАР-ТАСС Первин Мамед-заде/.   Минувший четверг стал для Ливии очередным днем кровавых   столкновений между противниками режима Муамара Каддафи и   лояльными ему силами безопасности.Желая перехватить инициативу и доказать свои все еще   значительные возможности по удержанию контроля над   ситуацией, власти в первой половине дня направили войска и   африканских наемников в расположенный в 50 км к западу от   Триполи город Эз-Зауия, который оказался в руках   восставших. В результате устроенной силовиками бойни с   использованием вертолетов и тяжелого оружия погибло   множество людей, были разрушены дома, мечети, школы. По   словам очевидцев, военные вели прицельный огонь по   безоружным жителям, явно намереваясь убивать, а не   запугивать. В общей сложности, утверждает спутниковый   телеканал "Аль-Джазира", в результате устроенной верными   Каддафи силами многочасовой резни погибли около 100   ч

Возникают ошибки. Где-то нет ссылок. Найдём проблемные новости и удалим их.

In [14]:
tss_urls = [ ]
for item in tass_urls:
    try:
        len(item[2])
        tss_urls.append(item)
    except:
        print(item)

['Все вологодские лагеря проверят после отравления детей под Череповцом', None, None, False, False]
['Обвиняемый в хищении 330 млн у Кержакова просит проверить нетрудоспособность футболиста', None, None, False, False]


In [15]:
tass_urls = tss_urls

Прямой подход

    i = 0
    nws = [ ]
    for url in tass_urls:
        nws.append(href_grab(url))
        i+=1
        if i % 10 == 0:
            with open('tass_news_'+str(i)+'.pickle', 'wb') as f:
                pickle.dump(nws, f)

Добавим немного паралельности в наш код

In [16]:
from multiprocessing.pool import ThreadPool

In [17]:
# Функция Map для нелинейной части. Она берёт какой-то вектор с данными и скачивает всю инфу по нему
def Map(vect):
    wect = [href_grab(item) for item in vect]
    return(wect)

# Функция Reduce для нелинейной части. Она берёт все результаты разных Map и объединяет их в единый вектор
def Reduce(l):
    ll = [ ]
    for item in l:
        ll.extend(item)
    return(ll)

# Функция, которая делит входной поток данных на несколько частей
def Separator(vect,part):
    n = len(vect)
    vec_parts = [round(n/part)*i for i in range(part)]
    vec_parts.append(n)
    prereduce = [vect[vec_parts[i]:vec_parts[i+1]] for i in range(part)]
    return(prereduce)

# Тестер того коректным ли было разбиение (на всякий случай)
def Test(begin,end):
    m=[]
    for item in end:
        m.extend(item)
    print(m==begin)

In [18]:
# Функция для выгрузки! Ура! Ура! Ура! 
def Kachatel(chto_kachat,chislo_potokov):
    # Map-шаг. В скобках у Pool задаётся количество потоков для работы 
    separatorlist = Separator(chto_kachat,chislo_potokov)
    Test(chto_kachat,separatorlist)
    # Качаем
    pool = ThreadPool(chislo_potokov)
    l = pool.map(Map, separatorlist)
    # Reduce-шаг.
    itog = Reduce(l)
    return(itog)

In [ ]:
# Делай два - распаралеленная часть.
# Ставим в аргументы вектор из линейной части и число потоков
# Результаты тестирования времени и количества потоков: 
# 1000 статей 5 потоков по 200 - 219 секунд (около 6 минут)
# 2000 статей 10 потоков по 200 -221 секунда
# 2000 статей 20 потоков по 100 -160 секунд (2 минуты)
# 20000 статей 20 потоков по 1000 штук - 1526 секунд (около 25 минут)
# 60000 cтатей 30 потоков              - 3467 секунд (2016-09-07)

In [19]:
len(tass_urls)

648381

Судя по всему есть ограничения на количество запросов. Экспериментальным путём было получено, что уместно будет качать в 30 потоков. Будем качать батчами по 100 000 каждые сутки по два захода. Через три дня у нас будут все тексты. 

In [18]:
tass_urls_tst = tass_urls[500000:]

In [19]:
%%time
data_vector = Kachatel(tass_urls_tst,30)

True
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error

In [20]:
errors = [item for item in  data_vector  if item[2] == 'Error' ]
len(errors)

219

In [21]:
len(data_vector)

148381

In [22]:
with open('news_tass_600+.pickle', 'wb') as f:
    pickle.dump(data_vector, f)

## 3. Проверка ошибок после скачки 

In [25]:
all_news = [ ]
for name in ['news_tass_100','news_tass_200','news_tass_300','news_tass_400','news_tass_500','news_tass_600+']:
    with open(name+'.pickle', 'rb') as f:
        nws = pickle.load(f)
    all_news.append(nws)

In [24]:
for nw in all_news:
    print(len([item for item in  nw  if item[2] == 'Error' ]),len(nw))

2994 100000
4190 100000
350 100000
277 100000
351 100000
219 148381


## 4. Лемматизация 

In [37]:
# теперь обработаем всё это дело токенайзером и лемматизатором

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def token_lemma(statia):
    out_1 = tokenizer.tokenize(statia.lower())
    # Возможно, имеет смысл нормировать статью на количество слов
    out_2 = [morph.parse(tx)[0].normal_form for tx in out_1]
    return out_2

In [41]:
token_lemma(all_news[0][0][0])

['париж', 'поддерживать', 'резолюция', 'сб', 'оон', 'по', 'перемирие', 'сирия']

In [43]:
lemm_all_news = [ ]
for nws in tqdm_notebook(all_news):
    cur_lemm_nws = [ ]
    for item in tqdm_notebook(nws):
        name_lemm = token_lemma(item[0])
        text_lemm = token_lemma(item[1])
        cur_lemm_nws.append([name_lemm, text_lemm, item[2],item[3],item[4],item[5],item[6],item[7]])
    lemm_all_news.append(cur_lemm_nws)    

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

HBox(children=(IntProgress(value=0, max=100000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=100000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=100000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=100000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=100000), HTML(value='')))

HBox(children=(IntProgress(value=0, max=148381), HTML(value='')))

In [44]:
for name,nws in zip(['100','200','300','400','500','600+'],lemm_all_news):
    with open('lemm_news_tass_' + name + '.pickle', 'wb') as f:
        pickle.dump(nws,f)